<div style="font-family: 'Source Code Pro'; font-size: 24px;">

## Maquinas virtuales

Hasta hace no muchos anios era comun que las empresas tuvieran un servidor por cada aplicativo importante que tubieran y muchas veces tenian directamente servidores independientes. La realidad es que como normalmente el proyecto involucraba comprar un sistema nuevo con requerimientos nuevos, entonces era comun que al encarar un proyecto tambien se involucraba al area de IT para que disponibilice la plataforma donde se iba a correr. Para algunas empresas, lo podian obviar, otras, cuando uno da un servicio y ese servicio esta destinado a una determinada empresa entonces si podia requerir que sea un servidor especifico. Esto le traia problemas al area de IT, lo primero que los data centers son caros en cuanto a lugar, alquilar un rac en un datacenter es caro porque viene con muchas cosas como seguridad, refrigeracion, etc. Toda esa infraesctructra es cara. Con la redundancia ante fallas lo que pasaba era que en primera instancia se caia el servicio, la gente de IT tenia que tener partes de reemplazo, pero si los distintos servidores eran de distintas tecnologias entonces no iba a servir el reemplazo. La otra alternativa era tener servidores completos de reemplazo pero ocuapn lugar y presentan un costo el tenerlos. Un tercer problema era que muchas veces habian recursos propios desaprovechados. Esto es lo que de alguna forma nos va a introducir al concepto de maquinas virtuales. Esto no es algo nuevo, si bien es algo que hizo IBM en la epoca del 60, cayo en desuso y con los problemas vistos anteriores la gente comenzo a ver que no eran tan mala idea. Cuando sacan la IBM S/360 que fue la primera la capacidad de RAM era de 4k. La idea era radicalmente distinta, en vez de correr un unico programa en un unico SO, y ese SO le presentaba las disitnas aplicaciones dispositivos virtuales. La idea es que en vez de un programa llamado SO virtualize a la aplicacion los dispositivos, tenemos otro programa que se llama virtualizador que le virtualiza el hardware a los SO. Entonces, voy a tener por ejemplo un teclado o un mouse y lo voy a distribuir entre esos distintos programas que estan corriendo que cada uno sera llamado maquina virtual. Que cosas deberiamos virtualizar? hay que virtualizar la RAM, tal vez es una de las mas facil de virtualizar, hay que dividirla entre los distintos candidatos, hay que virtualizar los procesadores, cosa que antes estrictamente hacia el SO repartiendo los procesadores entre los threads, ahora cada SO va a tratar de identificiar el procesador que tiene y cuantos cores tiene. Teclado, mouse, placa de video, todo esto tiene que ser hardware virtual. Entonces cada SO va a tener abajo un hardware pensando que es suyo cuando en realidad lo tiene compartido con otros. Este programa que permite crear estas diferentes instancias de hardwaare en cada una de las cuales podre instalar un SO se lo denomina hipervisor. Es como una vista superior al SO. Como queda la estructura ahora? esta virtualizacion del hardware hace que cada uno de los SO vea su propio hardware, ahora teemos la parte fisica y en vez de tenenr el SO sobre la parte fiisca como teniamos antes con sus drivers que manejaban el hardware y a su vez el SO con sus tareas, tenemos sobre el fisico el hipervisor que expone n instancias del hardware llamadas maquinas virtuales y en cada maquina virtual voy a instalar un SO y cada uno de esos SO correran un conjunto de tareas. Es como que agregamos una capa mas. Obviament estas maquinas pueden tener SO distintos. De la misma manera que el SO veia sus propios recuross y protegia a las tareas entre si, el hipervisor hace lo mismo pero entre SOs. El hipervisor hace que cada SO piense que su hardware es exclusivo como el SO hace lo mismo con sus tareas. Y el hipervisor es el que mantiene el orden para que no haya conflicto en el hardware. Esto da un aprovechameinto mucho mas eficiente de los recursos. Cuando un SO no tiene carga o nada que hacer le da control a los otros, y en el mismo hardware puedo correr distintos tipos de aplicaciones. Cuantas mas maquinas correr meor aprovechamiento. 

## Soporte de Hardware

Cada SO piensa que es su hardware, que nadie mas esta compartiendo nada con el, entocnes pretende usarlo de forma directa. De la misma forma que antes teniamos un task swithc cuando habia que sacarle a una tarea el procesador para darselo a otra, lo mismo va a tener que hacer el hipervisor, sacarle el procesador a uno para darselo a otro. Cambiar de una maquina a otra involucraria todo un tema en cuanto a segmentos, por otro lado como hago para que la maquina no use memoria que le dio a otra, se necesita soporte de hardware. Obviamente para no tener que estar entendiendo cada instruccion que cada SO quiere ejecutar, volvemos a crear hardware para detectar esas cosas. Ya hace tiempo que los procesadores tienen hardware especifico para las maquinas virtuales, estan reservados para un hipervisor que pueda controlar los distintos SOs. El hipervisor pasa a correr en ring 0, los SOs en ring 1 y las tareas en ring 3. Los fabricantes hace rato incluyeron diferentes tecnologias en los procesadores. Para que sirven estas nueva instrucciones? lo que era AMD-v o VT-x, escencialmente estan apuntando a tener toda la informacion de una maquina virtual, toda la informacion de sus segmentos de sus tablas de paginado, toda la configuracion de los permisos metidas en estructuras de forma que cuando el hipervisor quiera cambiar de una maquina a la otra, simplemente pueda cambiar un puntero y todas esas estructuras pasena a ser las nuevas de la otra maquina, no tengo que copiar a la misma zona todos los datos, simplemente cambio el puntero y lo hago mas rapido. Se crean algunas instrucciones nuevas, todas estan asociadas a mantenimiento de maquinas virtuales. Hay una instruccion que le permite a una VM llamar al virtualizador, es el equivalente a cuando queriamos que una tarea llame al SO con una instruccion. Entonces, se crea una instruccion que es un punto de entrada al hipervisor, tiene algunos usos interesantes. Segunda funcionalidad fundamental, el tema del paginado, el paginado son un monton de tablas, si cada vez que tenemos que conmutar de una maquina a la otra tenemos que rescribir todas las tablas llevaria mucho tiempo. Tanto intel como AMD, lo que hacen es crear un nuevo nivel al paginado que el SO no ve pero que se puede conmutar con las instrucciones anteriores, automaticamente cuando cambio de VM se pasa a usar otro juego de tablas sin necesidad de tener que recargar las tablas. La tercer funcionalidad es importante porque el hipervisor lo que hace es lograr que un hardware unico lo vea, es el acceso a los puertos de I/O. Lo que hace esto es permitir un acceso muy finito. 

## Paravirtualized Hardware:

Si bien los hipervisores pueden emular el hardware y "mentirle" a los SOs que piensan que le mandan comandos a los chips cuando en realidad son interpretados por el hipervisor, hay una forma mas eficiente.. Algunos hipervisores, admeas de virtualizar el hardware, le presentan hardware al SO que en realidad no existe, pero que soportan instrucciones de alto nivel. Un caso tipico son los adapatadores de red. Cada adaptador (de cada marca) tiene un chip distinto que soporta ciertos comandos. Cada adaptador requiere un driver. El driver, cuando el SO quiere enviar un paquete por la red, le carga el paquete al chip, le calcula el framing/checksum y luego lo transmite. El adaptador paravirtualizado se presenta como un chp de mas alto nivel. El driver del SO para este chip no tiene que instruir al chip que haga todas esas tareas, porque ese chip inteligente toma el paquete y hace todo el (en realidad lo hace el hipervisor).

## Over assignment

De la misma forma que podemos dar mas recursos puedo crear dos maquinas virtuales decirles que tienen 6 cores cada una y en realidad tengo 8 cores en total, obviamnete el hipervisor tendra que repartir los cores entre las maquinas virtuales. El proceso de dar mas recursos que los disponibles se llama over-assignment. Para algunas cosas esto funciona pero no para la memoria, un SO cuando no esta usando el disco no hace nada, no lo toca, entonces el virtualizador se da cuenta que no se quiere utilizar, que pasa cuando una tarea no imprime nada en pantalla, no escribe nada, ahora con la memoria, es un problema aparte. Los SOs utilizan toda la memoria, la tengan usada o no, a medida que las tareas las piden se la dan, el SO se va guardando todo en RAM, ese cache ocupa toda la memoria posible, eso hace que cada maquina use toda la RAM, y si todas las maquinas usan todas las RAM se complica el tema del over-assignment porque el over-assignment funciona cuando algunos lo usan en algun momento y otros en otro asi podemos repartir sin perdida de performance, cuando todos lo quieren utilizar a la vez la performance se degrada mucho porque perdemos mucho tiempo en el cambio de contexto. Ahi si hay un problema con el tema de la memoria porque todos los SOs tratan de utilizar toda la memoria, por supuesto a las tareas les dan la que piden pero todo lo que sobra lo guardan como cache de distintas cosas. Como se puede resolver esto? cuando uno carga una VM le carga un driver mas, ballon driver. La idea de este ballon driver es que el virtualizador necesita memoria, y tiene que ver a que SO se la puede sacar, el tema es a quien se la saca, como toma la decision. La realidad es que el virtualizador no sabe para que utiliza la memoria el SO. Entonces llama al driver y se fija si hay ram libre o si esta siendo utilizada por tareas, en el caso de encontrar un SO que este utilizando memoria como cache entonces ese SO es candidato a que se le saque memoria. Para hacer eso, el ballon driver le pide al SO un bloque de memoria, borra cosas de cache, y apenas el SO recibe la memoria la libera dejando al SO con un crocket libre. De esta forma el virtualizador puede logar que los distintos SOs huespedes bajen la cache. El over-assignment, bien usado, optmiza los recursos y permite tener la misma performance que con servidores individuales a un menor costo. 

## Docker

Estrictamente no es un hipervisor, apunta a otro concepto. En realidad apunta a que cuando uno tiene que distribuir una aplicacion no solamente manda el codigo de la aplicacion, cuando uno quiere distribuir muy sencillo para el usuario pedirle muchos requisitos y condiciones es demsiado. Docker lo que permite es generar un contenedor y distribuir en una unica imagen todo junto, distribuyo una parte del SO con toda la aplicacion instalada configurada, etc. No es necesariamente muy optimo ni que se use en un ambiente de data centers para correr aplicaciones en ambiente empresarial, pero es algo que muchas veces cuando uno tiene que mandarle algo a un usuario que no es de alto nivel es una buena alternativa. La imagen contiene todo, inclusive el SO. Por este motivo, una aplicacion distribuida con Docker puede ejectuarse en cualquier SO soportado sin cambios. Una funcionalidad util a los efectos de la seguridad es que se deben exponer explicitamente las funcionalidades que seran visibles desde fuera de la aplicacion. Por ejemplo, si corremos un servidor HTTP dentro de la aplicacion y se desea que sea accesible desde otras aplicaciones que corren en el mismo servidor, se lo debe declarar expresamente. La imagen no solo tiene la aplicacion y el SO, tambiene tiene el FileSystem, firewall, etc. De esta forma puede distribuirse todo en uno. 

## Configuracion de VM

Lo primero que se hace es declarar que SO le voy a poner. Uno puede elegir cuanta memoria darle. El sistema normalmente calcula en funcion de la memoria real que tiene el sistema fisico. El orden de booteo se carga en el bios. Hay dos formas de arranque, la forma de boot sector o el EFI. Todos los SOs tienen un chip que se llama reloj de tiempo real. La realidad es que algunos SOs como linux mandan la hora en UTC, y despues con la banda horaria muestran la hora local. Para que no se empiecen a pelear los SOs hay que decirle como quiero la hora. Procesadores, se pueden asignar los cores arbitrariamente entre todos. La sobrealocacion es algo que depende del uso de las distintas maquinas. Pantalla, cuanta memoria de video le queremos dar, cuantos monitores y el controlador grafico. El controlador grafico es el chip que va a emular de video, en defnitica el SO va a tener que cargar un driver para el hardware. Almacenamiento, en realidad los discos de las maquinas tienen dos alternativas: uno es que sea en un archivo del sitema de archivos del hipervisor, todo lo que un determinado SO esta escribiendo en su disco enrealidad escribe en su archivo. La otra alternativa es crear un archivo que contiene todo, en los datacenters es un volumen del storage. Audio, para emularlo el controlador emula un audio, incluso hasta con la opcion de tenerlo o no. Adaptadores de red, permite pedir el chip que queres emular, aparte de emular el chip, cuando una maquina virtual quiera sacar algo por la red, probablmente el hipervisor tenga varias placas de red fisicas y yo quiera que algunas vms hablen por una y otras por otra, entonces tengo distintas formas de conectar ese chip a la red. USB, el tema de USB es mas complejo porque mas alla de tener el controlador uno a veces quiere enchufar dos dispositivos y quiero que uno la vea una maquina y el otro otra maquina, entonces hay un menu que permite asignar un USB a una maquina determinada. En definitiva el objetivo de la configuracion de la maquina virtual es seleccionar el hardware, que hardware se le expondra al SO. Como hacemos con el acceso? tenemos un monton de maquinas virtuales adentro de nuestro servidor fisico que puede ser usado por todos en la empresa, como hacemos con los usuarios? las aplicaciones son pensadas para uso remoto entonces desde un lugar remoto abro una sesion y hago lo que tengo que hacer.

## Granjas de maquinas virtuales

Hay hipervisores que permiten trabajar en conjunto en granjas. Pones varios host fisicos, en todos instalamos la misma marca de hipervisor y con eso pongo todas las maquinas virtuales. Los hipervisores se pueden hablar entre si. Que ventaja nos da que los hipervisores que corren en distintos hosts fisicos se puedan hablar entre si? si un hipervisor por ejemplo detecta que esta con mucha carga de CPU, que esta con muchas tareas corriendo a la vez, puede tomar algunas acciones, la mas importante es que puede mudar la maquina en tiempo real a que se siga ejecutando en otro hipervisor. Esto no solamente nos da inmunidad ante fallas sino que tambien seguimos mejorando el aprovechamiento de los recursos, unas de las vetnajas que teniamos de poner varias maquinas en un mismo servidor fisico era que como no todas las maquinas virtuales utilizan todos los recursos a la vez, el hipervisor podia balancear y aprovechar mejor el host fisico. Ahora podemos hacer un aprovechamiento entre servidores. Las granjas de maquinas virtuales mudan la maquina completo a otro servidor fisico migrando una "foto" de la memoria de la VM y los datos que el hipervisor tiene de esa VM (solo algunos GB). A parte de lo que es la foto de la RAM con los registros la aplicacion puede tener conexiones de red abierta y sobretodo puede tener conexiones con los discos, entonces ese si seria un problema, estaria el problema de acceder a los archivos previos. Entonces cuando se trabaja con esta arquitectura no se tienen los discos fisicamente en un servidor, en definitiva un disco es una conexion de red. Esta posibilidad de migrar maquinas en caliente es lo que se llama Machine-Migration y se produce de forma transparente al SO. Ya que tenemos esta posibilidad de pasar las maquinas mientras se ejecutan de un lado a otro, esto podemos no solo usarlo por un tema de balance de CPU, si se llega a detectar un problema siempre el hipervisor tiene la posibilidad de mover eso a otra maquina fisica y sacar las maquinas del servidor que tiene problemas. El servidor con problemas puede repararse sin que los usuarios noten el problema. Esta funcionalidad por este motivo tambien es conocida como HA (High Availability).